# Chapter 3: Symmetry Operations and Point Groups

## Learning Objectives
- Understand the fundamental symmetry operations in chemistry
- Implement symmetry operations as matrices
- Classify molecules by their point groups
- Use symmetry to generate equivalent atomic positions
- Understand the connection between symmetry and physical properties

---

## 3.1 Introduction to Molecular Symmetry

Symmetry is fundamental to chemistry and materials science:
- **Reduces computation**: We only need to calculate for the asymmetric unit
- **Predicts properties**: Selection rules, degeneracies, allowed transitions
- **Classifies molecules**: Point groups provide a systematic classification
- **Determines crystal structures**: Space groups describe periodic symmetry

### The Five Types of Symmetry Elements

| Symbol | Name | Description |
|--------|------|-------------|
| $E$ | Identity | Do nothing (every molecule has this) |
| $C_n$ | Proper rotation | Rotation by $360°/n$ around an axis |
| $\sigma$ | Mirror plane | Reflection through a plane |
| $i$ | Inversion center | Inversion through a point |
| $S_n$ | Improper rotation | Rotation by $360°/n$ followed by reflection |

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import List, Tuple, Optional, Dict, Callable
from dataclasses import dataclass
from enum import Enum

# Reuse classes from previous chapters
class Atom:
    ELEMENTS = {
        'H':  {'Z': 1,  'mass': 1.008,   'color': 'white',  'radius': 0.31},
        'C':  {'Z': 6,  'mass': 12.011,  'color': 'gray',   'radius': 0.76},
        'N':  {'Z': 7,  'mass': 14.007,  'color': 'blue',   'radius': 0.71},
        'O':  {'Z': 8,  'mass': 15.999,  'color': 'red',    'radius': 0.66},
        'F':  {'Z': 9,  'mass': 18.998,  'color': 'green',  'radius': 0.57},
        'Cl': {'Z': 17, 'mass': 35.453,  'color': 'lime',   'radius': 1.02},
        'S':  {'Z': 16, 'mass': 32.065,  'color': 'yellow', 'radius': 1.05},
    }
    
    def __init__(self, symbol: str, position: np.ndarray):
        self.symbol = symbol
        self.position = np.array(position, dtype=float)
        props = self.ELEMENTS.get(symbol, {'Z': 0, 'mass': 1.0, 'color': 'gray', 'radius': 1.0})
        self.atomic_number = props['Z']
        self.mass = props['mass']
        self.color = props['color']
        self.covalent_radius = props['radius']

class Structure:
    def __init__(self, name: str = "unnamed"):
        self.name = name
        self.atoms: List[Atom] = []
    
    def add_atom(self, symbol: str, position: np.ndarray) -> None:
        self.atoms.append(Atom(symbol, position))
    
    def __len__(self) -> int:
        return len(self.atoms)
    
    def get_positions(self) -> np.ndarray:
        return np.array([atom.position for atom in self.atoms])
    
    def set_positions(self, positions: np.ndarray) -> None:
        for atom, pos in zip(self.atoms, positions):
            atom.position = pos
    
    def get_symbols(self) -> List[str]:
        return [atom.symbol for atom in self.atoms]
    
    def copy(self) -> 'Structure':
        new_struct = Structure(self.name + "_copy")
        for atom in self.atoms:
            new_struct.add_atom(atom.symbol, atom.position.copy())
        return new_struct
    
    def get_center_of_mass(self) -> np.ndarray:
        total_mass = sum(atom.mass for atom in self.atoms)
        return sum(atom.mass * atom.position for atom in self.atoms) / total_mass

print("Classes loaded successfully!")

## 3.2 Implementing Symmetry Operations

All symmetry operations can be represented as 3×3 matrices (or 4×4 for combined rotation+translation).

### The Identity Operation ($E$)

$$E = \begin{pmatrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \end{pmatrix}$$

### Proper Rotations ($C_n$)

Rotation by $\theta = 2\pi/n$ around an axis.

In [ ]:
class SymmetryOperation:
    """Base class for symmetry operations."""
    
    def __init__(self, name: str, matrix: np.ndarray):
        self.name = name
        self.matrix = matrix
    
    def apply(self, position: np.ndarray) -> np.ndarray:
        """Apply the symmetry operation to a position."""
        return self.matrix @ position
    
    def apply_to_structure(self, structure: Structure) -> Structure:
        """Apply the symmetry operation to all atoms in a structure."""
        new_struct = structure.copy()
        positions = new_struct.get_positions()
        new_positions = (self.matrix @ positions.T).T
        new_struct.set_positions(new_positions)
        return new_struct
    
    def __repr__(self) -> str:
        return f"SymmetryOp({self.name})"
    
    def __matmul__(self, other: 'SymmetryOperation') -> 'SymmetryOperation':
        """Combine two symmetry operations."""
        return SymmetryOperation(
            f"{self.name}·{other.name}",
            self.matrix @ other.matrix
        )

def identity() -> SymmetryOperation:
    """Create the identity operation E."""
    return SymmetryOperation("E", np.eye(3))

def rotation_Cn(n: int, axis: np.ndarray = np.array([0, 0, 1])) -> SymmetryOperation:
    """Create a Cn rotation operation.
    
    Args:
        n: Order of rotation (C2, C3, C4, etc.)
        axis: Rotation axis (default: z-axis)
    
    Returns:
        SymmetryOperation for Cn rotation
    """
    theta = 2 * np.pi / n
    axis = axis / np.linalg.norm(axis)
    kx, ky, kz = axis
    c, s = np.cos(theta), np.sin(theta)
    t = 1 - c
    
    R = np.array([
        [c + kx*kx*t,      kx*ky*t - kz*s,  kx*kz*t + ky*s],
        [ky*kx*t + kz*s,   c + ky*ky*t,     ky*kz*t - kx*s],
        [kz*kx*t - ky*s,   kz*ky*t + kx*s,  c + kz*kz*t]
    ])
    
    return SymmetryOperation(f"C{n}", R)

def mirror_plane(normal: np.ndarray) -> SymmetryOperation:
    """Create a mirror plane (reflection) operation.
    
    Args:
        normal: Normal vector to the mirror plane
    
    Common planes:
        σh (horizontal): normal = rotation axis
        σv (vertical): normal perpendicular to rotation axis
        σd (dihedral): bisects angle between C2 axes
    """
    n = normal / np.linalg.norm(normal)
    sigma = np.eye(3) - 2 * np.outer(n, n)
    
    # Determine plane type based on normal
    if np.allclose(np.abs(n), [0, 0, 1]):
        name = "σh"
    else:
        name = "σv"
    
    return SymmetryOperation(name, sigma)

def inversion_center() -> SymmetryOperation:
    """Create the inversion operation i."""
    return SymmetryOperation("i", -np.eye(3))

def improper_rotation_Sn(n: int, axis: np.ndarray = np.array([0, 0, 1])) -> SymmetryOperation:
    """Create an Sn improper rotation (roto-reflection).
    
    Sn = Cn followed by σh (reflection through perpendicular plane)
    """
    Cn = rotation_Cn(n, axis)
    sigma_h = mirror_plane(axis)
    combined = sigma_h.matrix @ Cn.matrix
    return SymmetryOperation(f"S{n}", combined)

# Create some common operations
E = identity()
C2_z = rotation_Cn(2, [0, 0, 1])
C3_z = rotation_Cn(3, [0, 0, 1])
C4_z = rotation_Cn(4, [0, 0, 1])
sigma_h = mirror_plane([0, 0, 1])  # xy-plane
sigma_v = mirror_plane([1, 0, 0])  # yz-plane
i = inversion_center()
S4_z = improper_rotation_Sn(4, [0, 0, 1])

print("Symmetry operations created:")
print(f"E (identity): \n{E.matrix}\n")
print(f"C2 (180° rotation around z): \n{np.round(C2_z.matrix, 10)}\n")
print(f"C3 (120° rotation around z): \n{np.round(C3_z.matrix, 6)}\n")
print(f"σh (horizontal mirror, xy-plane): \n{sigma_h.matrix}\n")
print(f"i (inversion): \n{i.matrix}")

## 3.3 Verifying Symmetry Operations

### Key Mathematical Properties

1. **Closure**: Combining two operations gives another operation in the group
2. **Identity**: $E \cdot g = g \cdot E = g$ for any operation $g$
3. **Inverse**: Every operation has an inverse ($g \cdot g^{-1} = E$)
4. **Order**: $g^n = E$ for some integer $n$ (the order of the operation)

### Determinants
- **Proper rotations**: $\det(R) = +1$
- **Improper operations** (reflections, inversions, Sn): $\det(R) = -1$

In [ ]:
def analyze_symmetry_operation(op: SymmetryOperation) -> dict:
    """Analyze properties of a symmetry operation."""
    M = op.matrix
    
    # Determinant
    det = np.linalg.det(M)
    
    # Find the order (smallest n such that M^n = I)
    order = 1
    power = M.copy()
    while order < 20:  # Safety limit
        if np.allclose(power, np.eye(3)):
            break
        power = power @ M
        order += 1
    
    # Is it proper (rotation) or improper?
    is_proper = det > 0
    
    # Find fixed axis (eigenvector with eigenvalue 1)
    eigenvalues, eigenvectors = np.linalg.eig(M)
    fixed_axis = None
    for i, ev in enumerate(eigenvalues):
        if np.isclose(ev, 1.0):
            fixed_axis = eigenvectors[:, i].real
            break
    
    return {
        'name': op.name,
        'determinant': det,
        'order': order,
        'is_proper': is_proper,
        'type': 'proper rotation' if is_proper else 'improper operation',
        'fixed_axis': fixed_axis
    }

# Analyze our operations
operations = [E, C2_z, C3_z, C4_z, sigma_h, i, S4_z]

print("Analysis of Symmetry Operations:")
print("-" * 60)
for op in operations:
    info = analyze_symmetry_operation(op)
    print(f"{info['name']:6s} | det={info['determinant']:+.0f} | order={info['order']:2d} | {info['type']}")

# Verify: C3^3 = E
print("\nVerification: C3 @ C3 @ C3 = E?")
C3_cubed = C3_z @ C3_z @ C3_z
print(C3_cubed.name)
print(f"Matrix: \n{np.round(C3_cubed.matrix, 10)}")

## 3.4 Generating Equivalent Positions

A powerful application of symmetry: starting from one atom, generate all symmetry-equivalent positions.

In [ ]:
def generate_equivalent_positions(position: np.ndarray, 
                                   operations: List[SymmetryOperation],
                                   tolerance: float = 1e-6) -> np.ndarray:
    """Generate all symmetry-equivalent positions.
    
    Args:
        position: Starting position
        operations: List of symmetry operations
        tolerance: Tolerance for considering positions identical
    
    Returns:
        Array of unique equivalent positions
    """
    equivalent = [position.copy()]
    
    for op in operations:
        new_pos = op.apply(position)
        
        # Check if this position is already in the list
        is_duplicate = False
        for existing in equivalent:
            if np.linalg.norm(new_pos - existing) < tolerance:
                is_duplicate = True
                break
        
        if not is_duplicate:
            equivalent.append(new_pos)
    
    return np.array(equivalent)

# Example: Generate a benzene ring using C6 symmetry
# Start with one carbon
C_start = np.array([1.4, 0.0, 0.0])  # C-C aromatic distance ≈ 1.4 Å

# C6 generates 6 equivalent positions: E, C6, C6^2, C6^3, C6^4, C6^5
C6 = rotation_Cn(6, [0, 0, 1])
C6_operations = [E]
current = E
for i in range(5):
    current = current @ C6
    C6_operations.append(current)

C_positions = generate_equivalent_positions(C_start, C6_operations)

print(f"Generated {len(C_positions)} carbon positions:")
for i, pos in enumerate(C_positions):
    print(f"C{i+1}: {np.round(pos, 4)}")

# Visualize
fig, ax = plt.subplots(figsize=(8, 8))
ax.scatter(C_positions[:, 0], C_positions[:, 1], s=500, c='gray', edgecolors='black')
for i, pos in enumerate(C_positions):
    ax.annotate(f'C{i+1}', pos[:2], ha='center', va='center', fontsize=10)

# Draw bonds
for i in range(6):
    j = (i + 1) % 6
    ax.plot([C_positions[i, 0], C_positions[j, 0]], 
            [C_positions[i, 1], C_positions[j, 1]], 'k-', linewidth=2)

ax.set_aspect('equal')
ax.set_title('Benzene carbons generated by C6 symmetry')
ax.set_xlabel('X (Å)')
ax.set_ylabel('Y (Å)')
ax.grid(True, alpha=0.3)
plt.show()

## 3.5 Point Groups

A **point group** is the complete set of symmetry operations for a molecule. All operations share at least one common point (usually the center of mass).

### Common Point Groups

| Point Group | Symmetry Elements | Examples |
|-------------|-------------------|----------|
| $C_1$ | E only | CHFClBr |
| $C_s$ | E, σ | HOCl |
| $C_i$ | E, i | meso-tartaric acid |
| $C_n$ | E, Cn | H2O2 (C2) |
| $C_{nv}$ | E, Cn, nσv | H2O (C2v), NH3 (C3v) |
| $C_{nh}$ | E, Cn, σh | trans-C2H2Cl2 (C2h) |
| $D_n$ | E, Cn, nC2' | allene (D2) |
| $D_{nh}$ | E, Cn, nC2', σh, nσv | benzene (D6h), BF3 (D3h) |
| $D_{nd}$ | E, Cn, nC2', nσd, S2n | staggered ethane (D3d) |
| $T_d$ | 24 operations | CH4 |
| $O_h$ | 48 operations | SF6 |

In [ ]:
class PointGroup:
    """A molecular point group with its symmetry operations."""
    
    def __init__(self, name: str, operations: List[SymmetryOperation]):
        self.name = name
        self.operations = operations
    
    def __repr__(self) -> str:
        return f"PointGroup({self.name}, {len(self.operations)} operations)"
    
    def order(self) -> int:
        """Return the order (number of operations) of the point group."""
        return len(self.operations)
    
    def generate_equivalent_structure(self, structure: Structure) -> Structure:
        """Generate all symmetry-equivalent atoms."""
        result = Structure(f"{structure.name}_{self.name}")
        
        for atom in structure.atoms:
            equiv_positions = generate_equivalent_positions(
                atom.position, self.operations
            )
            for pos in equiv_positions:
                # Check if this position already exists
                is_duplicate = False
                for existing in result.atoms:
                    if (existing.symbol == atom.symbol and 
                        np.linalg.norm(existing.position - pos) < 0.01):
                        is_duplicate = True
                        break
                if not is_duplicate:
                    result.add_atom(atom.symbol, pos)
        
        return result

def create_C2v() -> PointGroup:
    """Create the C2v point group.
    
    Operations: E, C2, σv(xz), σv(yz)
    Example molecule: H2O
    """
    E = identity()
    C2 = rotation_Cn(2, [0, 0, 1])
    sigma_xz = mirror_plane([0, 1, 0])  # xz-plane (y=0)
    sigma_yz = mirror_plane([1, 0, 0])  # yz-plane (x=0)
    
    return PointGroup("C2v", [E, C2, sigma_xz, sigma_yz])

def create_C3v() -> PointGroup:
    """Create the C3v point group.
    
    Operations: E, 2C3, 3σv
    Example molecule: NH3
    """
    E = identity()
    C3 = rotation_Cn(3, [0, 0, 1])
    C3_2 = SymmetryOperation("C3²", C3.matrix @ C3.matrix)  # C3 squared
    
    # Three σv planes at 120° intervals
    sigma_v1 = mirror_plane([1, 0, 0])
    sigma_v2 = mirror_plane([np.cos(np.pi/3), np.sin(np.pi/3), 0])
    sigma_v3 = mirror_plane([np.cos(2*np.pi/3), np.sin(2*np.pi/3), 0])
    
    return PointGroup("C3v", [E, C3, C3_2, sigma_v1, sigma_v2, sigma_v3])

def create_D6h() -> PointGroup:
    """Create the D6h point group (benzene symmetry).
    
    Order = 24 operations
    """
    operations = [identity()]
    
    # C6 rotations: C6, C6^2=C3, C6^3=C2, C6^4=C3^2, C6^5
    C6_mat = rotation_Cn(6, [0, 0, 1]).matrix
    power = np.eye(3)
    for i in range(1, 6):
        power = power @ C6_mat
        operations.append(SymmetryOperation(f"C6^{i}", power))
    
    # 6 C2' axes in the plane
    for i in range(6):
        angle = i * np.pi / 6
        axis = np.array([np.cos(angle), np.sin(angle), 0])
        C2_prime = rotation_Cn(2, axis)
        C2_prime.name = f"C2'_{i+1}"
        operations.append(C2_prime)
    
    # σh (horizontal mirror)
    sigma_h = mirror_plane([0, 0, 1])
    sigma_h.name = "σh"
    operations.append(sigma_h)
    
    # i (inversion)
    operations.append(inversion_center())
    
    # S6, S3 operations
    S6 = improper_rotation_Sn(6, [0, 0, 1])
    S3 = improper_rotation_Sn(3, [0, 0, 1])
    operations.extend([S6, S3])
    
    # 6 σv planes
    for i in range(6):
        angle = i * np.pi / 6
        normal = np.array([-np.sin(angle), np.cos(angle), 0])
        sigma_v = mirror_plane(normal)
        sigma_v.name = f"σv_{i+1}"
        operations.append(sigma_v)
    
    return PointGroup("D6h", operations)

def create_Td() -> PointGroup:
    """Create the Td point group (tetrahedral, e.g., CH4).
    
    Order = 24 operations
    Elements: E, 8C3, 3C2, 6S4, 6σd
    """
    operations = [identity()]
    
    # Tetrahedral vertices (normalized)
    vertices = np.array([
        [1, 1, 1], [1, -1, -1], [-1, 1, -1], [-1, -1, 1]
    ], dtype=float)
    vertices = vertices / np.linalg.norm(vertices[0])
    
    # 8 C3 operations (2 for each body diagonal, clockwise and counter-clockwise)
    for v in vertices:
        C3 = rotation_Cn(3, v)
        C3_inv = SymmetryOperation(f"C3⁻¹", np.linalg.inv(C3.matrix))
        operations.extend([C3, C3_inv])
    
    # 3 C2 operations (through edge midpoints)
    C2_axes = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    for axis in C2_axes:
        C2 = rotation_Cn(2, axis)
        operations.append(C2)
    
    # 6 S4 operations (along coordinate axes)
    for axis in C2_axes:
        S4 = improper_rotation_Sn(4, axis)
        S4_3 = SymmetryOperation("S4³", np.linalg.matrix_power(S4.matrix, 3))
        operations.extend([S4, S4_3])
    
    # 6 σd planes (containing pairs of opposite edges)
    diag_planes = [
        [1, 1, 0], [1, -1, 0], [1, 0, 1], [1, 0, -1], [0, 1, 1], [0, 1, -1]
    ]
    for normal in diag_planes:
        sigma_d = mirror_plane(normal)
        sigma_d.name = "σd"
        operations.append(sigma_d)
    
    # Remove duplicates
    unique_ops = []
    for op in operations:
        is_dup = False
        for existing in unique_ops:
            if np.allclose(op.matrix, existing.matrix):
                is_dup = True
                break
        if not is_dup:
            unique_ops.append(op)
    
    return PointGroup("Td", unique_ops)

# Create and display point groups
C2v = create_C2v()
C3v = create_C3v()
D6h = create_D6h()
Td = create_Td()

print("Point Groups:")
print(f"{C2v} - Example: H2O")
print(f"{C3v} - Example: NH3")
print(f"{D6h} - Example: Benzene")
print(f"{Td} - Example: CH4")

## 3.6 Building Molecules Using Symmetry

Let's use point groups to build molecules from minimal information.

In [ ]:
def build_water_C2v() -> Structure:
    """Build H2O using C2v symmetry.
    
    In C2v, we only need to specify one H atom - 
    the other is generated by the C2 rotation.
    """
    # Create minimal structure with O at origin and one H
    water_asym = Structure("H2O_asymmetric_unit")
    water_asym.add_atom('O', [0, 0, 0])
    
    # H position: 0.96 Å from O, at angle 52.25° from +x axis
    # (half of 104.5° H-O-H angle)
    oh_length = 0.96
    half_angle = np.radians(104.5 / 2)
    water_asym.add_atom('H', [oh_length * np.cos(half_angle), 
                              oh_length * np.sin(half_angle), 0])
    
    # Apply C2v symmetry
    return C2v.generate_equivalent_structure(water_asym)

def build_ammonia_C3v() -> Structure:
    """Build NH3 using C3v symmetry."""
    # N at origin, one H
    nh3_asym = Structure("NH3_asymmetric_unit")
    nh3_asym.add_atom('N', [0, 0, 0])
    
    # H position
    nh_length = 1.02
    hnh_angle = np.radians(107)  # H-N-H angle
    # Calculate position
    h_z = -nh_length * np.cos(np.radians(180 - hnh_angle/2))
    h_r = nh_length * np.sin(np.radians(180 - hnh_angle/2))
    nh3_asym.add_atom('H', [h_r, 0, h_z])
    
    return C3v.generate_equivalent_structure(nh3_asym)

def build_methane_Td() -> Structure:
    """Build CH4 using Td symmetry."""
    ch4_asym = Structure("CH4_asymmetric_unit")
    ch4_asym.add_atom('C', [0, 0, 0])
    
    # One H in the [1,1,1] direction
    ch_length = 1.09
    h_pos = np.array([1, 1, 1]) / np.sqrt(3) * ch_length
    ch4_asym.add_atom('H', h_pos)
    
    return Td.generate_equivalent_structure(ch4_asym)

# Build and display
water = build_water_C2v()
ammonia = build_ammonia_C3v()
methane = build_methane_Td()

print(f"Water: {len(water)} atoms (expected 3)")
for atom in water.atoms:
    print(f"  {atom.symbol}: {np.round(atom.position, 4)}")

print(f"\nAmmonia: {len(ammonia)} atoms (expected 4)")
for atom in ammonia.atoms:
    print(f"  {atom.symbol}: {np.round(atom.position, 4)}")

print(f"\nMethane: {len(methane)} atoms (expected 5)")
for atom in methane.atoms:
    print(f"  {atom.symbol}: {np.round(atom.position, 4)}")

In [ ]:
# Visualize the molecules
def plot_structure_3d(structure: Structure, ax, title: str):
    """Plot a structure on a 3D axis."""
    for atom in structure.atoms:
        ax.scatter(*atom.position, c=atom.color, s=atom.covalent_radius*500, 
                   edgecolors='black', linewidths=1)
    ax.set_xlabel('X (Å)')
    ax.set_ylabel('Y (Å)')
    ax.set_zlabel('Z (Å)')
    ax.set_title(title)

fig = plt.figure(figsize=(15, 5))

ax1 = fig.add_subplot(131, projection='3d')
plot_structure_3d(water, ax1, 'H₂O (C₂ᵥ)')

ax2 = fig.add_subplot(132, projection='3d')
plot_structure_3d(ammonia, ax2, 'NH₃ (C₃ᵥ)')

ax3 = fig.add_subplot(133, projection='3d')
plot_structure_3d(methane, ax3, 'CH₄ (Tᵈ)')

plt.tight_layout()
plt.show()

## 3.7 Detecting Molecular Symmetry

Given a molecule, how do we determine its point group? This is a classic algorithm.

In [ ]:
def is_structure_symmetric(structure: Structure, op: SymmetryOperation, 
                            tolerance: float = 0.1) -> bool:
    """Check if a structure has a given symmetry operation.
    
    The structure is symmetric if applying the operation maps each atom
    to an equivalent atom (same element, same position).
    """
    positions = structure.get_positions()
    symbols = structure.get_symbols()
    
    # Apply operation to all positions
    transformed = (op.matrix @ positions.T).T
    
    # Check that each transformed position matches an original atom
    for i, (new_pos, symbol) in enumerate(zip(transformed, symbols)):
        found_match = False
        for j, (orig_pos, orig_sym) in enumerate(zip(positions, symbols)):
            if symbol == orig_sym and np.linalg.norm(new_pos - orig_pos) < tolerance:
                found_match = True
                break
        if not found_match:
            return False
    
    return True

def find_principal_axis(structure: Structure) -> Tuple[np.ndarray, int]:
    """Find the principal rotation axis and its order.
    
    Returns:
        Tuple of (axis direction, highest Cn order)
    """
    # Test common axis directions
    test_axes = [
        [0, 0, 1], [0, 1, 0], [1, 0, 0],
        [1, 1, 0], [1, 0, 1], [0, 1, 1],
        [1, 1, 1]
    ]
    
    best_axis = np.array([0, 0, 1])
    best_order = 1
    
    for axis in test_axes:
        axis = np.array(axis, dtype=float)
        axis = axis / np.linalg.norm(axis)
        
        # Test for C2, C3, C4, C6
        for n in [6, 4, 3, 2]:
            Cn = rotation_Cn(n, axis)
            if is_structure_symmetric(structure, Cn):
                if n > best_order:
                    best_order = n
                    best_axis = axis
    
    return best_axis, best_order

def detect_point_group(structure: Structure) -> str:
    """Attempt to detect the point group of a structure.
    
    This is a simplified algorithm that handles common cases.
    """
    # Center the structure at center of mass
    centered = structure.copy()
    com = centered.get_center_of_mass()
    positions = centered.get_positions() - com
    centered.set_positions(positions)
    
    # Check for inversion
    has_inversion = is_structure_symmetric(centered, inversion_center())
    
    # Find principal axis
    axis, n = find_principal_axis(centered)
    
    # Check for horizontal mirror
    has_sigma_h = is_structure_symmetric(centered, mirror_plane(axis))
    
    # Check for vertical mirrors
    perp_axes = []
    if np.allclose(axis, [0, 0, 1]):
        perp_axes = [[1, 0, 0], [0, 1, 0]]
    else:
        # Find perpendicular axis
        perp = np.cross(axis, [0, 0, 1])
        if np.linalg.norm(perp) < 0.1:
            perp = np.cross(axis, [1, 0, 0])
        perp_axes = [perp / np.linalg.norm(perp)]
    
    has_sigma_v = False
    for perp in perp_axes:
        if is_structure_symmetric(centered, mirror_plane(perp)):
            has_sigma_v = True
            break
    
    # Check for perpendicular C2 axes
    has_C2_perp = False
    for perp in perp_axes:
        if is_structure_symmetric(centered, rotation_Cn(2, perp)):
            has_C2_perp = True
            break
    
    # Determine point group (simplified logic)
    if n == 1:
        if has_inversion:
            return "Ci"
        elif has_sigma_v:
            return "Cs"
        else:
            return "C1"
    
    if has_C2_perp:
        # D groups
        if has_sigma_h:
            return f"D{n}h"
        elif has_sigma_v:
            return f"D{n}d"
        else:
            return f"D{n}"
    else:
        # C groups
        if has_sigma_h:
            return f"C{n}h"
        elif has_sigma_v:
            return f"C{n}v"
        else:
            return f"C{n}"

# Test on our molecules
print("Point Group Detection:")
print(f"Water: {detect_point_group(water)}")
print(f"Ammonia: {detect_point_group(ammonia)}")
print(f"Methane: {detect_point_group(methane)}")

## 3.8 Symmetry and the Moment of Inertia Tensor

The **moment of inertia tensor** is intimately connected to molecular symmetry. It's always diagonal in the principal axis frame, and the principal axes are determined by symmetry.

$$I = \begin{pmatrix} I_{xx} & I_{xy} & I_{xz} \\ I_{xy} & I_{yy} & I_{yz} \\ I_{xz} & I_{yz} & I_{zz} \end{pmatrix}$$

Where:
$$I_{\alpha\beta} = \sum_i m_i (|\mathbf{r}_i|^2 \delta_{\alpha\beta} - r_{i,\alpha} r_{i,\beta})$$

In [ ]:
def moment_of_inertia_tensor(structure: Structure) -> np.ndarray:
    """Calculate the moment of inertia tensor.
    
    The structure should be centered at its center of mass.
    
    Returns:
        3x3 moment of inertia tensor in amu·Å²
    """
    # Center at COM
    com = structure.get_center_of_mass()
    
    I = np.zeros((3, 3))
    
    for atom in structure.atoms:
        r = atom.position - com
        m = atom.mass
        r_sq = np.dot(r, r)
        
        for alpha in range(3):
            for beta in range(3):
                if alpha == beta:
                    I[alpha, beta] += m * (r_sq - r[alpha]**2)
                else:
                    I[alpha, beta] -= m * r[alpha] * r[beta]
    
    return I

def principal_moments(structure: Structure) -> Tuple[np.ndarray, np.ndarray]:
    """Calculate principal moments of inertia and principal axes.
    
    Returns:
        Tuple of (eigenvalues, eigenvectors) where eigenvalues are
        the principal moments (Ia, Ib, Ic) in ascending order.
    """
    I = moment_of_inertia_tensor(structure)
    eigenvalues, eigenvectors = np.linalg.eigh(I)
    
    # Sort by ascending eigenvalue
    idx = np.argsort(eigenvalues)
    
    return eigenvalues[idx], eigenvectors[:, idx]

def classify_rotor(structure: Structure, tolerance: float = 0.01) -> str:
    """Classify the molecular rotor type based on moments of inertia.
    
    Returns:
        'linear', 'spherical top', 'symmetric top', or 'asymmetric top'
    """
    Ia, Ib, Ic = principal_moments(structure)[0]
    
    # Normalize by the largest moment
    max_I = max(Ic, 1e-10)
    Ia_norm = Ia / max_I
    Ib_norm = Ib / max_I
    Ic_norm = Ic / max_I
    
    if Ia_norm < tolerance:  # Ia ≈ 0
        return "linear"
    elif np.isclose(Ia_norm, Ic_norm, rtol=tolerance):
        return "spherical top"
    elif np.isclose(Ia_norm, Ib_norm, rtol=tolerance) or np.isclose(Ib_norm, Ic_norm, rtol=tolerance):
        return "symmetric top"
    else:
        return "asymmetric top"

# Analyze our molecules
print("Molecular Rotor Classification:")
print("-" * 60)

for name, mol in [('Water', water), ('Ammonia', ammonia), ('Methane', methane)]:
    moments, axes = principal_moments(mol)
    rotor_type = classify_rotor(mol)
    
    print(f"\n{name}:")
    print(f"  Principal moments (amu·Å²): {np.round(moments, 4)}")
    print(f"  Rotor type: {rotor_type}")
    print(f"  Point group correlation: ", end="")
    if rotor_type == "spherical top":
        print("High symmetry (Td, Oh, Ih)")
    elif rotor_type == "symmetric top":
        print("Cn axis with n ≥ 3 (C3v, D3h, etc.)")
    elif rotor_type == "asymmetric top":
        print("Low symmetry (C2v, Cs, C1, etc.)")
    elif rotor_type == "linear":
        print("Infinite rotational symmetry (C∞v, D∞h)")

---

## Practice Exercises

### Exercise 3.1: Verify Group Closure
For the C2v point group, verify that any combination of two operations gives another operation in the group.

In [ ]:
# YOUR CODE HERE
def verify_closure(point_group: PointGroup) -> bool:
    """Verify that the point group satisfies closure.
    
    For every pair of operations g1 and g2, their product g1·g2
    must also be in the group.
    """
    # TODO: Implement
    pass

### Exercise 3.2: Build SF6 with Oh Symmetry
Sulfur hexafluoride has Oh symmetry (octahedral). Build it using symmetry operations.

In [ ]:
# YOUR CODE HERE
def build_SF6() -> Structure:
    """Build SF6 using Oh symmetry.
    
    S-F bond length: 1.56 Å
    The S is at the center, with F atoms along ±x, ±y, ±z.
    """
    # TODO: Implement
    pass

### Exercise 3.3: Character Table Entry
Calculate the trace (character) of each symmetry operation in C2v.

In [ ]:
# YOUR CODE HERE
def compute_character_table(point_group: PointGroup) -> dict:
    """Compute the character (trace) of each operation.
    
    The trace of a rotation by θ is: 1 + 2cos(θ)
    The trace of a reflection is: 1
    The trace of inversion is: -3
    """
    # TODO: Implement
    pass

### Exercise 3.4: Cyclopentadienyl Ring
Build a cyclopentadienyl ring (C5H5) using C5v symmetry.

In [ ]:
# YOUR CODE HERE
def build_cyclopentadienyl() -> Structure:
    """Build C5H5 (cyclopentadienyl) using C5v symmetry.
    
    C-C bond length: 1.40 Å
    C-H bond length: 1.08 Å
    """
    # TODO: Implement
    pass

### Exercise 3.5: Symmetry Orbit
Write a function that computes the "orbit" of an atom under a point group - all positions it visits under all operations.

In [ ]:
# YOUR CODE HERE  
def symmetry_orbit(position: np.ndarray, point_group: PointGroup) -> np.ndarray:
    """Compute all symmetry-equivalent positions.
    
    This is the "orbit" of the position under the point group.
    The number of unique positions divides the order of the group.
    """
    # TODO: Implement
    pass

---

## Key Takeaways

1. **Five symmetry elements**: E, Cn, σ, i, Sn cover all molecular symmetry
2. **All operations are matrices**: Rotations have det=+1, improper operations have det=-1
3. **Point groups classify molecules** by their complete set of symmetry operations
4. **Symmetry reduces computation**: Only need to specify the asymmetric unit
5. **Symmetry determines properties**: Moments of inertia, selection rules, etc.

## Next Chapter Preview

In Chapter 4, we'll explore **Crystal Lattices and Unit Cells**:
- Bravais lattices
- Lattice vectors and parameters
- Fractional vs Cartesian coordinates
- The reciprocal lattice